In [1]:
from fmm import FastMapMatch,Network,NetworkGraph,UBODTGenAlgorithm,UBODT,FastMapMatchConfig
from fmm import Network,NetworkGraph,STMATCH,STMATCHConfig
from shapely.geometry import LineString,shape, mapping

In [2]:
# Read network and graph
network = Network("network-Porto/edges.shp","fid", "u", "v")
print "Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count())
graph = NetworkGraph(network)

Nodes 908 edges 1675


In [3]:
# Precompute UBODT
# Can be skipped if you already generated an ubodt file
ubodt_gen = UBODTGenAlgorithm(network,graph)
# The delta is defined as 3 km approximately. 0.03 degrees. 
status = ubodt_gen.generate_ubodt("network-Porto/ubodt.txt", 0.03, binary=False, use_omp=True)
# Binary is faster for both IO and precomputation
# status = ubodt_gen.generate_ubodt("stockholm/ubodt.bin", 0.03, binary=True, use_omp=True)
print status

Status: success
Time takes 0.519 seconds



In [3]:
# Create FMM model
### Read UBODT
ubodt = UBODT.read_ubodt_csv("network-Porto/ubodt.txt")
### Create FMM model
fmm_model = FastMapMatch(network,graph,ubodt)

k = 8
radius = 0.003
gps_error = 0.0005
fmm_config = FastMapMatchConfig(k,radius,gps_error)

In [4]:
import os, glob
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm

In [35]:
a = np.zeros((8760, 3350))
b = [[] for row in range(1675)]
start = datetime.datetime(2013, 7, 1, 0, 0, 0)
# FMM
for t in tqdm(range(13)):
    df = pd.read_csv('data/tra' + str(t) + '.csv')
    for wkt, ti in zip(df['geometry'], df['TIMESTAMP']):

        result = fmm_model.match_wkt(wkt, fmm_config)

        candidates = []
        for c in result.candidates:
            candidates.append((c.edge_id,c.source,c.target,c.error,c.length,c.offset,c.spdist,c.ep,c.tp))

        df_c = pd.DataFrame(candidates, columns=["eid","source","target","error","length","offset","spdist","ep","tp"])
        df_c['speed'] = df_c['spdist'] * 24000
        
        now_time = ti
        if now_time.count(':') == 1:
            now_time += ':00'
        cnow_time = datetime.datetime.strptime(now_time, "%Y-%m-%d %H:%M:%S")
        
        for i in range(len(df_c) - 1):
            if i != 0 or i != len(df_c) - 1:
                if df_c['eid'][i] != df_c['eid'][i+1]:
                    if df_c['speed'][i] >= 5 and df_c['speed'][i] < 120:
                        b[int(df_c['eid'][i])].append(str(round(df_c['speed'][i], 2)))
                        index = int((((cnow_time - start).total_seconds()) / 60) // 60)
                        a[index][int(df_c['eid'][i])*2] += 1
                        a[index][int(df_c['eid'][i])*2 + 1] += df_c['speed'][i]
            cnow_time += datetime.timedelta(seconds=15)

100%|██████████| 13/13 [1:50:39<00:00, 510.74s/it]


In [36]:
tf = pd.DataFrame(a)
tf.to_csv('data/traffic_info.csv', index=False)
tf

,0,1,2,3,4,5,6,7,8,9,...,3340,3341,3342,3343,3344,3345,3346,3347,3348,3349
0,2.0,39.429934,0.0,0.000000,4.0,133.672792,1.0,34.812037,1.0,34.725719,...,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,26.834133,0.0,0.0
1,2.0,50.572508,1.0,31.917500,2.0,61.683941,1.0,12.877057,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,2.0,34.161755,0.0,0.0
2,3.0,67.625311,0.0,0.000000,2.0,36.999047,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.0,0.000000,0.0,0.000000,2.0,56.697334,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
4,1.0,35.033288,0.0,0.000000,1.0,17.791063,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
5,0.0,0.000000,1.0,32.601434,0.0,0.000000,1.0,25.077050,1.0,35.120948,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
6,1.0,11.824647,1.0,25.689052,1.0,25.082700,1.0,30.703425,0.0,0.000000,...,0.0,0.0,1.0,20.666313,0.0,0.000000,0.0,0.000000,0.0,0.0
7,0.0,0.000000,1.0,31.865496,2.0,43.124539,4.0,59.959848,1.0,25.464566,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
8,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,2.0,32.530532,...,0.0,0.0,3.0,125.771667,0.0,0.000000,1.0,24.828332,0.0,0.0
9,0.0,0.000000,2.0,35.742970,1.0,19.069289,2.0,29.166364,0.0,0.000000,...,0.0,0.0,9.0,273.175458,0.0,0.000000,0.0,0.000000,0.0,0.0


In [37]:
spd = np.zeros((8760, 1675))
volume = np.zeros((8760, 1675))
for i in tqdm(range(8760)):
    for j in range(1675):
        spd[i, j] = tf.loc[i, j*2+1] / tf.loc[i, j*2]
        volume[i, j] = tf.loc[i, j*2]

  0%|          | 0/8760 [00:00<?, ?it/s]/home/skang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
100%|██████████| 8760/8760 [05:02<00:00, 29.00it/s]


In [38]:
speed = pd.DataFrame(spd)
vol = pd.DataFrame(volume)
speed.to_csv('data/speed.csv', index=False)
vol.to_csv('data/volume.csv', index=False)

In [52]:
limit = np.zeros(1675)
for i in range(1675):
    temp = sorted(b[i])
    if len(temp) >= 6:
        lspeed = temp[int(round(len(temp) * 0.9))]
    elif len(temp) > 0:
        lspeed = temp[len(temp)-1]
    else:
        lspeed = 0
    limit[i] = lspeed

In [54]:
df = pd.DataFrame(limit)
df

,0
0,43.45
1,7.93
2,46.63
3,54.14
4,7.22
5,39.89
6,44.91
7,7.92
8,84.66
9,93.69


In [55]:
df.to_csv('data/limitSpeed.csv')

In [57]:
df2 = pd.DataFrame(b)
df2

,0,1,2,3,4,5,6,7,8,9,...,118714,118715,118716,118717,118718,118719,118720,118721,118722,118723
0,32.26,15.29,21.53,25.69,35.28,13.84,13.74,35.03,11.82,31.83,...,None,None,None,None,None,None,None,None,None,None
1,31.92,32.6,25.69,31.87,25.49,10.26,7.5,28.19,22.58,7.54,...,None,None,None,None,None,None,None,None,None,None
2,33.97,37.18,30.09,31.59,23.44,31.82,29.32,27.38,13.55,30.71,...,None,None,None,None,None,None,None,None,None,None
3,12.88,34.81,25.08,30.7,26.91,15.09,5.88,12.07,18.39,10.78,...,None,None,None,None,None,None,None,None,None,None
4,34.73,35.12,25.46,24.36,8.17,29.81,29.42,18.56,15.72,30.22,...,None,None,None,None,None,None,None,None,None,None
5,33.78,27.26,40.38,37.75,28.52,15.99,35.27,20.54,31.45,28.31,...,None,None,None,None,None,None,None,None,None,None
6,43.45,45.83,35.66,48.43,17.89,39.76,15.54,42.63,18.79,38.41,...,None,None,None,None,None,None,None,None,None,None
7,54.98,25.4,38.7,31.27,22.55,34.5,35.53,35.32,36.51,21.92,...,None,None,None,None,None,None,None,None,None,None
8,52.35,50.18,33.64,69.1,37.38,28.28,8.59,76.47,80.12,58.16,...,None,None,None,None,None,None,None,None,None,None
9,61.09,111.64,96.08,58.89,60.28,34.24,119.8,68.42,35.72,36.62,...,None,None,None,None,None,None,None,None,None,None


In [59]:
df2 = df2.T

In [60]:
df2.to_csv('data/AllSpeedData.csv', index=False)